In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/vijayg15/Machine-Learning-project-with-MLflow-deployment.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "vijayg15"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "032a6763f4e1199604dda938e424da596b24ccba"

In [4]:
#set MLFLOW_TRACKING_URI = https://dagshub.com/vijayg15/Machine-Learning-project-with-MLflow-deployment.mlflow
#set MLFLOW_TRACKING_USERNAME = vijayg15
#set MLFLOW_TRACKING_PASSWORD = 032a6763f4e1199604dda938e424da596b24ccba

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/vijayg15/Machine-Learning-project-with-MLflow-deployment.mlflow",
           
        )

        return model_evaluation_config


In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn


In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self, actual, pred):
        acc = accuracy_score(actual, pred)
        prec = precision_score(actual, pred)
        rec = recall_score(actual, pred)
        cm = confusion_matrix(actual, pred)
        cr = classification_report(actual, pred)
        return acc, prec, rec, cm, cr
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(X_test)

            (acc, prec, rec, cm, cr) = self.eval_metrics(y_test, predicted_qualities)
            
            # Saving metrics as local
            #scores = {"Accuracy": acc, "Precision": prec, "Recall": rec, "Confusion Mat": cm, "C_report": cr}
            scores = {"Accuracy": acc, "Precision": prec, "Recall": rec, "Confusion Mat": np.array(cm).tolist()}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metrics({"Accuracy": acc})
            mlflow.log_metric("Precision", prec)
            mlflow.log_metric("Recall", rec)
            #mlflow.log_metric("Classification report", cr)

            mlflow.log_dict(np.array(cm).tolist(), "confusion_matrix.json")

            disp = ConfusionMatrixDisplay(confusion_matrix=cm)
            disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
            plt.savefig("ConfusionMatrix.png")
            mlflow.log_artifact("ConfusionMatrix.png")
            plt.close()


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-07-08 23:02:06,894: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-08 23:02:06,901: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-08 23:02:06,906: INFO: common: yaml file: schema.yaml loaded successfully]


[2024-07-08 23:02:06,908: INFO: common: created directory at: artifacts]
[2024-07-08 23:02:06,910: INFO: common: created directory at: artifacts/model_evaluation]
